## Group Project: EPA FE Analysis and Prediction

### Context: 


### Objective : 

* To identify the different factors that affect fuel economy in surveyed vehicles
* To make a model to predict if an employee will attrite or not


### Dataset :
The data contains (replace - demographic details), (replace - work-related metrics) and (replace - attrition flag).

* **Metric** - Definition


### Import the necessary libraries

In [9]:
# Note: go through later and removed unused dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

#algorithms to use
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

#for tuning the model
from sklearn.model_selection import GridSearchCV

#to ignore warnings
import warnings
warnings.filterwarnings("ignore")

### Read the dataset

In [21]:
#reading the dataset

# for use if you want to narrow the number of columns in use
usecols = ['Manufacturer','Model Year','Vehicle Type', '2-Cycle MPG']

# use the commented out df definition to import only the columns specified in the above list
df = pd.read_csv('table_export.csv')
# df = pd.read_csv('table_export.csv', usecols = usecols, sep = ',')
df.head()

,Manufacturer,Model Year,Regulatory Class,Vehicle Type,Production (000),Production Share,2-Cycle MPG,Real-World MPG,Real-World MPG_City,Real-World MPG_Hwy,...,Multivalve Engine,Variable Valve Timing,Average Number of Gears,Transmission - CVT (Non-Hybrid),4 or Fewer Gears,5 Gears,6 Gears,7 Gears,8 Gears,9 or More Gears
0,BMW,1975,All,All,20,0.002,17.23631,14.69368,13.38272,16.6922,...,-,-,-,-,0.578,-,-,-,-,-
1,BMW,1975,Car,All Car,20,0.002,17.23631,14.69368,13.38272,16.6922,...,-,-,-,-,0.578,-,-,-,-,-
2,BMW,1975,Truck,All Truck,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,BMW,1975,Car,Car SUV,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,Ford,1975,Truck,All Truck,644,0.064,12.54248,10.67381,9.92732,11.75409,...,-,-,-,-,-,-,-,-,-,-


### Dataset Info

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170 entries, 0 to 5169
Data columns (total 54 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   Manufacturer                                         5170 non-null   object
 1   Model Year                                           5170 non-null   object
 2   Regulatory Class                                     5170 non-null   object
 3   Vehicle Type                                         5170 non-null   object
 4   Production (000)                                     5170 non-null   object
 5   Production Share                                     5170 non-null   object
 6   2-Cycle MPG                                          5170 non-null   object
 7   Real-World MPG                                       5170 non-null   object
 8   Real-World MPG_City                                  5170 non-null   object
 9

## Exploratory Data Analysis and Data Preprocessing